# 1. 使用langchain

In [ ]:
!pip install langchain

# 2. 导入第三方库

In [1]:
import os
import sys
import tarfile

from erniebot_agent.agents import RetrievalAgent
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import WholeMemory
from erniebot_agent.retrieval import BaizhongSearch
from erniebot_agent.tools.baizhong_tool import BaizhongSearchTool
from erniebot_agent.tools.langchain_retrieval_tool import LangChainRetrievalTool
from langchain.docstore.document import Document
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import SpacyTextSplitter
from erniebot_agent.utils.common import download_file

使用aistudio access token, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [2]:
aistudio_access_token = "<your aistudio access token>"
os.environ["EB_AGENT_ACCESS_TOKEN"] = aistudio_access_token
os.environ["AISTUDIO_ACCESS_TOKEN"] = aistudio_access_token

# 3. 预处理

## 3.1 下载数据集

In [3]:
if not os.path.exists("construction_regulations"):
    file_path = "construction_regulations.tar"
    download_file(
        url="https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar",
        save_path=file_path,
    )

    files = tarfile.open(file_path, "r:*")
    file_dir = os.path.dirname(file_path)
    files.extractall(file_dir, files.getmembers())
# ! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
# ! tar xvf construction_regulations.tar

## 3.2 构建知识库

In [4]:
embeddings = ErnieEmbeddings()

In [5]:
faiss_name = "faiss_index"
if os.path.exists(faiss_name):
    db = FAISS.load_local(faiss_name, embeddings)
else:
    loader = PyPDFDirectoryLoader("construction_regulations")
    documents = loader.load()
    text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=320, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_name)

# 4. 基于复杂Query分解的RetrievalAgent
## 4.1 复杂Query任务规划
在实际的问答场景，用户提出的Query往往是复杂的，比如用户问:"请比较一下波士顿和姚明出生的城市的区别是什么？"，这个问题涉及到了两个城市，并且还要比较两个城市的区别，如果直接把这个问题当成检索的Query，如果库里面没有直接回答这个问题的答案，那么这个问题就不能被正确检索，导致召回的结果与该Query的相关度偏低，最后就不能被正确的回复。

解决复杂Query的比较好的方法就是把复杂的Query分解成比较简单的子问题，比如上面的问题可以分解成：波士顿的人口，波士顿的GDP，姚明出生的城市，姚明出生城市的人口，姚明出生城市的GDP等。这些子问题一般都能够检索到正确的答案，最后大模型把这些子问题的答案进行汇总，得到这个query的最终的答案。

In [6]:
search_tool = LangChainRetrievalTool(db=db, threshold=0.0)
llm = ERNIEBot(model="ernie-3.5", api_type="aistudio")
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=False,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [7]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)
response

AgentResponse(text='城市设计管理办法的具体内容包括以下几点：\n\n1. 建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。\n2. 城市设计分为总体城市设计和重点地区城市设计。总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 编制城市设计时，应广泛征求专家和公众意见，审批前应依法进行公示，公示时间不少于30日。城市设计成果应当自批准之日起20个工作日内，通过政府信息网站以及当地主要新闻媒体予以公布。\n4. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n5. 城市、县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应当审核城市设计要求落实情况。同时，在开展城市规划实施评估时，应当同时评估城市设计工作实施情况。\n6. 城市设计的技术管理规定由国务院城乡规划主管部门另行制定。各地可根据本办法，按照实际情况，制定实施细则和技术导则。县人民政府所在地以外的镇可以参照本办法开展城市设计工作。\n7. 本办法自2017年6月1日起施行。\n\n以上内容仅供参考，如需获取更多信息，建议查阅《城市设计管理办法》原文件第四章“实施与监督”部分。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 有条件的地方可以建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台。   第七条 城市设计分为总体城市设计和重点地区城市设计。\n\n  \n\n第八条 总体城市设计应当确定城市风貌特色， 保护自然山水格局， 优化城市形态格局， 明确公共空间体系， 并可与城市 （县人民政府所在地建制镇）总体规划一并报批。\n\n    第2个段落: 第十三条 编制城市设计时，组织编制机关应当通过座谈、论证、网络等多种形式及渠道，广泛征求专家和公众意见。\n\n审批前应依法进行公示，公示时间不少于30日。\n\n  \n\n城市设计成果应当自批准之日起20个工作日内，通过政府信息网站以及当地主要新闻媒体予以公布。   \n\n第十四条 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到

## 4.2 使用compressor压缩子query检索结果

一个复杂的Query，往往需要分解成若干个子query，但是每个子query检索到的结果往往包含了很多不相关或者重复的信息，但这些信息会占用大量的Token，可能会导致大模型的token长度过长的情况。其中一个解决方法就是把每个子query检索到的信息进行压缩，即只提取与子query相关的片段，或者进行简短的总结即可，然后把总结或者提取的内容当成是子query的结果，这样会比直接使用子query的结果更好。

In [8]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [9]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)
response

AgentResponse(text='城市设计管理办法的具体内容包括：\n\n1. 城市设计分为总体城市设计和重点地区城市设计。\n2. 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 在编制城市设计时，组织编制机关应当通过座谈、论证、网络等多种形式及渠道，广泛征求专家和公众意见。审批前应依法进行公示，公示时间不少于30日。\n4. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 城市设计管理办法的目标是：\n1. 提高城市建设水平\n2. 塑造城市风貌特色\n3. 推进城市设计工作\n4. 完善城市规划建设管理\n\n    第2个段落: 城市设计管理办法涵盖的方面有：\n1. 确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系；\n2. 总体城市设计可与城市（县人民政府所在地建制镇）总体规划一并报批；\n3. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程；\n4. 城市、县人民政府城乡规划主管部门进行建筑设计方案审查和规划核实时，应审核城市设计要求落实情况；\n5. 城市、县人民政府城乡规划主管部门开展城市规划实施评估时，应同时评估城市设计工作实施情况；\n6. 城市设计的技术管理规定由国务院城乡规划主管部门另行制定；\n7. 各地可根据本办法，按照实际情况，制定实施细则和技术导则；\n8. 县人民政府所在地以外的镇可以参照本办法开展城市设计工作；\n9. 本办法自2017年6月1日起施行。\n\n    第3个段落: 城市设计管理办法的实施主要通过以下几个方面进行：\n\n1. 建立城市设计管理辅助决策系统，并将城市设计要求纳入城市规划管理信息平台，以实现城市设计的有效管理和监督。\n2. 城市设计分为总体城市设计和重点地区城市设计，总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 城市设计是落实城市规

## 4.3 使用few shot example 来引导子query分解

在利用大模型对复杂的Query分解成子query的时候，由于缺乏一些策略指导，大模型分解的子query会很随机，比如“请比较一下波士顿和姚明出生的城市的区别是什么？”，可能会被分解成：波士顿和姚明出生城市的人口，波士顿和姚明出生城市的GDP等。这些子Query虽然也比较简单具体，但不是最优的，不是我们预想的把两个城市拆开来生成子问题。因此，我们可以加入一些few shot样本来引导大模型按照我们的预想的方式来生成子问题，这样能够避免大模型出现很随机的子问题分解的情况。

In [10]:
data_map = {
    "电动汽车的品牌有哪些？各有什么特点？": [
        "当前市场上的主要电动汽车品牌。",
        "每个品牌的电动汽车品牌的基本技术规格，如续航里程、充电速度等。",
        "每个品牌的电动汽车型号，包括轿车、SUV、卡车等。",
        "每个品牌的充电基础设施和网络覆盖情况。",
        "每个品牌的电池技术和电动驱动系统。",
    ],
    "智能家居科技趋势及其在提高生活效率中的应用": ["当前最新的智能家居科技趋势是什么", "智能家居科技趋势的工作原理和特性", "哪些公司或产品正在引领这些智能家居科技趋势"],
    "人工智能技术在医疗诊断中的应用案例和效果分析": [
        "人工智能技术在医疗诊断领域中的主要应用案例",
        "收集每个应用案例中使用的具体人工智能技术，如深度学习、机器学习等",
        "分析这些应用案例中人工智能技术对诊断准确性的影响。",
        "研究人工智能技术在医疗诊断中的效果，特别关注是否提高了患者的治疗成功率。",
    ],
}

In [11]:
few_shot_examples = []
for key, values in data_map.items():
    meta_data = {}
    for i in range(len(values)):
        meta_data[f"sub_query_{i+1}"] = values[i]
    doc = Document(page_content=key, metadata={"sub_queries": meta_data})
    few_shot_examples.append(doc)

In [12]:
few_shot_retriever = FAISS.from_documents(few_shot_examples, embeddings)
few_shot_search = LangChainRetrievalTool(few_shot_retriever)

In [13]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    few_shot_retriever=few_shot_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [14]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)
response

/root/anaconda3/envs/py39/lib/python3.9/site-packages/langchain_core/vectorstores.py:311: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='智能家居科技趋势及其在提高生活效率中的应用', metadata={'sub_queries': {'sub_query_1': '当前最新的智能家居科技趋势是什么', 'sub_query_2': '智能家居科技趋势的工作原理和特性', 'sub_query_3': '哪些公司或产品正在引领这些智能家居科技趋势'}}), -0.07231259528132505), (Document(page_content='电动汽车的品牌有哪些？各有什么特点？', metadata={'sub_queries': {'sub_query_1': '当前市场上的主要电动汽车品牌。', 'sub_query_2': '每个品牌的电动汽车品牌的基本技术规格，如续航里程、充电速度等。', 'sub_query_3': '每个品牌的电动汽车型号，包括轿车、SUV、卡车等。', 'sub_query_4': '每个品牌的充电基础设施和网络覆盖情况。', 'sub_query_5': '每个品牌的电池技术和电动驱动系统。'}}), -0.10011265655911084), (Document(page_content='人工智能技术在医疗诊断中的应用案例和效果分析', metadata={'sub_queries': {'sub_query_1': '人工智能技术在医疗诊断领域中的主要应用案例', 'sub_query_2': '收集每个应用案例中使用的具体人工智能技术，如深度学习、机器学习等', 'sub_query_3': '分析这些应用案例中人工智能技术对诊断准确性的影响。', 'sub_query_4': '研究人工智能技术在医疗诊断中的效果，特别关注是否提高了患者的治疗成功率。'}}), -0.12608110009431006)]
  warnings.warn(


AgentResponse(text='城市设计管理办法的具体内容包括：\n\n1. 城市设计分为总体城市设计和重点地区城市设计。\n2. 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 编制城市设计时，应广泛征求专家和公众意见，并在审批前依法进行公示，公示时间不少于30日。\n4. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n5. 在历史文化街区和历史风貌保护相关控制地区开展城市设计时，应保护历史文化，明确新建建筑和改扩建建筑的控制要求。\n6. 在重要街道、街区开展城市设计时，应考虑交通组织、交通设施、市政设施、街道家具、步行活动和绿化空间，提升街道特色和活力。\n7. 在城市设计重点地区范围以外地区，可以根据当地实际条件，依据总体城市设计，单独或者结合控制性详细规划等开展城市设计，明确建筑特色、公共空间和景观风貌等方面的要求。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 城市设计管理办法的制定目的是：\n1. 提高城市建设水平\n2. 塑造城市风貌特色\n3. 推进城市设计工作\n4. 完善城市规划建设管理\n\n    第2个段落: 根据所提供的背景，城市设计管理办法包含的关键要素和规定有：\n\n1. 城市设计分为总体城市设计和重点地区城市设计。\n2. 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n3. 编制城市设计时，应广泛征求专家和公众意见，并在审批前依法进行公示，公示时间不少于30日。\n4. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n5. 在历史文化街区和历史风貌保护相关控制地区开展城市设计时，应保护历史文化，明确新建建筑和改扩建建筑的控制要求。\n6. 在重要街道、街区开展城市设计时，应考虑交通组织、交通设施、市政设施、街道家具、步行活动和绿化空间，提升街道特色和活力。\n7. 在城市设计重点地区范围以外地区，可以根据当地实际条件，依据总体城市设

## 4.4 使用先验背景信息来引导子query分解
在利用大模型对复杂的Query分解成子query的时候，由于缺少先验知识，即不知道知识库里面包含了哪些内容，因此在做query分解的时候，会生成一些无用的子query，比如：“请比较一下波士顿和姚明出生的城市的区别是什么？”，如果库里面没有对美食的论述，而分解子query的时候出现了：波士顿有哪些美食，姚明的城市有哪些美食。这类无用的query，占了搜索资源，又对最终答案无效，会造成一定的浪费。因此，我们使用context retriever的方式，在做复杂query分解的时候，根据检索到的上下文内容进行分解，这样可以避免很多无效的子query分解的问题。

In [15]:
background_data = [
    "《城市管理执法办法》主要规定了城市管理执法的范围、主管部门、执法人员资格与行为准则、执法措施与程序，以及监督与责任追究机制，旨在确保城市管理执法活动规范、公正、文明，维护城市公共秩序和市容环境。",
    "《城市设计管理办法》主要内容是规定了城市设计的编制、审批、实施和管理的相关要求，旨在通过科学、合理的城市设计，塑造城市特色风貌，提升城市环境质量，促进城市可持续发展。该办法强调了城市设计的综合性、整体性和长远性，明确了各级政府和相关部门在城市设计中的职责和作用，为城市设计工作的规范化、制度化提供了保障。",
]

In [16]:
background_info = []
for item in background_data:
    meta_data = {}
    doc = Document(page_content=item)
    background_info.append(doc)

In [17]:
context_retriever = FAISS.from_documents(background_info, embeddings)
context_search = LangChainRetrievalTool(context_retriever)

In [18]:
memory = WholeMemory()
agent = RetrievalAgent(
    knowledge_base=search_tool,
    context_retriever=context_search,
    use_compressor=True,
    llm=llm,
    top_k=3,
    tools=[],
    memory=memory,
)

In [19]:
query = "城市设计管理办法有哪些具体的内容？"
response = await agent.run(query)
response

AgentResponse(text='城市设计管理办法的具体内容包括：\n\n1. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n2. 城市设计分为总体城市设计和重点地区城市设计。\n3. 总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n4. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。', chat_history=[<HumanMessage role: 'user', content: '检索结果:\n\n    第1个段落: 以下是与问题相关的部分：\n\n1. 城市设计管理办法（2017年3月14日中华人民共和国住房和城乡建设部令第35号公布 自2017年6月1日起施行）\n2. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n3. 城市设计分为总体城市设计和重点地区城市设计。\n4. 总体城市设计应当确定城市风貌特色， 保护自然山水格局， 优化城市形态格局， 明确公共空间体系， 并可与城市 （县人民政府所在地建制镇）总体规划一并报批。\n5. 重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n\n    第2个段落: - 第七条：城市设计分为总体城市设计和重点地区城市设计。\n- 第八条：总体城市设计应当确定城市风貌特色，保护自然山水格局，优化城市形态格局，明确公共空间体系，并可与城市（县人民政府所在地建制镇）总体规划一并报批。\n- 第十四条：重点地区城市设计的内容和要求应当纳入控制性详细规划，并落实到控制性详细规划的相关指标中。\n\n检索语句: 城市设计管理办法有哪些具体的内容？\n请根据以上检索结果回答检索语句的问题', token_count: 315>, <AIMessage role: 'assistant', content: '城市设计管理办法的具体内容包括：\n\n1. 城市设计是落实城市规划、指导建筑设计、塑造城市特色风貌的有效手段，贯穿于城市规划建设管理全过程。\n2. 城市设计分为总体城市设计和重点地区城市设计。\n3. 总体城市设计应当确定